In [67]:
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [82]:
data = pd.read_csv("spotify_millsongdata.csv")

In [5]:
data.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
#Data Cleaning

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [8]:
data.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [1]:
data = data.drop('link', axis=1)
data = data.sample(10000)
data.reset_index(drop=True)

NameError: name 'data' is not defined

In [12]:
#Regex

In [84]:
data['text'] = data['text'].str.lower()
data['text'] = data['text'].replace(r'^\w\s\r', ' ').replace(r'\n', ' ', regex = True)

In [ ]:
#Tokenization and Stemming

In [85]:
#stemmer = PorterStemmer()
def tokenize(txt):
    tokens = word_tokenize(txt)
    lemma = [nltk.WordNetLemmatizer(i) for i in tokens]
    #stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(lemma)

In [86]:
data['text'] = data['text'].apply(lambda x: tokenize(x))

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/babs/nltk_data'
    - '/Users/babs/anaconda3/nltk_data'
    - '/Users/babs/anaconda3/share/nltk_data'
    - '/Users/babs/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(data['text'])
similarity = cosine_similarity(matrix)

In [80]:
data[data['song'] == 'Bang'].index[0]

21901

In [62]:
def recommendation(song_df):
    idx = data[data['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(data.iloc[m_id[0]].song)

    return songs



In [66]:
recommendation('Crying Over You')

['Bang A Drum',
 'Bang Bang',
 'Bang Bang',
 'Bang Bang',
 'Bang Bang Bang',
 'Bang Bang Bang',
 'Pop That Thang',
 'Big Bang Boogie',
 'Things Meant To Be',
 'Dust 2...',
 'I Never Meant To Hurt You',
 'Moon Is Blue',
 'Bang',
 "I'll Find My Way Home",
 'Only Tears',
 'Vodevil',
 'Act Of Love',
 'Handful Of Dust',
 'Paris St. Germain',
 'You Were Meant For Me']

In [65]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(data,open('df.pkl','wb'))